In [1]:
.open tmp3.db
.param set foo bar

.load ../../dist/vec0
.load ./lembed0

insert into lembed_models(name, model)
  values (
    'default',
    lembed_model_from_file('all-MiniLM-L6-v2.e4ce9877.q8_0.gguf')
  );

select vec_version(), lembed_version();

vec_version(),lembed_version()
v0.1.3-alpha.2,v0.0.1-alpha.7


## FTS Search

In [2]:
.param set query planned parenthood

select
  rowid,
  highlight(fts_headlines, 0, '<b>', '</b>') as headline_highlighted,
  rank
from fts_headlines
where headline match :query
order by rank
limit 10;

rowid,headline_highlighted,rank
4666,Kamala Harris visits <b>Planned</b> <b>Parenthood</b> clinic,-18.9139950477264
6521,Former Marine sentenced to 9 years in prison for firebombing <b>Planned</b> <b>Parenthood</b> clinic,-14.807022703838651


##  Vector Search

In [3]:
.param set query planned parenthood

select
  article_id,
  articles.headline,
  distance
from vec_headlines
left join articles on articles.rowid = vec_headlines.article_id
where headline_embedding match lembed(:query)
  and k = 10;

article_id,headline,distance
9475,Inside a Gen Z campaign to shake up Congress,1.0655490159988403
11236,Bunnie XO and Jelly Roll announce plans to have a baby using IVF,1.071580171585083
4110,First over-the-counter birth control pill hits store shelves,1.0831280946731567
1808,More nonpregnant women are requesting abortion pills to have on hand,1.0897283554077148
11154,Meet the anti-abortion group using white coats and research to advance its cause,1.089759349822998
5980,Deathbed confession leads to bodies of mother and daughter killed 24 years ago,1.092090368270874
3540,"Holding photos of their deceased children, parents lobby Congress to pass online safety legislation",1.1028637886047363
4109,Alabama fertility clinic at the center of IVF ruling is resuming some procedures,1.103183627128601
4889,"Inside the organized crime rings plaguing Ulta, T.J. Maxx, Walgreens and other retailers",1.1056489944458008
86,Florida bars transgender people from changing the sex on their driver's licenses,1.1073163747787476


## keyword-first

In [15]:
.param set query abortion bans


.param set weight_fts 1.0
.param set weight_vec 1.0
.param set rrf_k 60
.param set k 10


with vec_matches as (
  select
    article_id,
    row_number() over (order by distance) as rank_number,
    distance as score
  from vec_headlines
  where
    headline_embedding match lembed(:query)
    and k = :k
  order by distance
),
fts_matches as (
  select
    rowid as article_id,
    row_number() over (order by rank) as rank_number,
    rank as score
  from fts_headlines
  where headline match :query
  limit :k
),
combined as (
  select 'fts' as match_type, * from fts_matches
  union all
  select 'vec' as match_type, * from vec_matches
),
final as (
  select
    articles.id,
    articles.headline,
    combined.*
  from combined
  left join articles on articles.rowid = combined.article_id
)
select * from final;



id,headline,match_type,article_id,rank_number,score
10098,Kamala Harris says abortion bans are creating 'a health care crisis',fts,10098,1,-10.678829270936067
9776,"States with abortion bans saw birth control prescriptions fall post-Dobbs, study finds",fts,9776,2,-10.016316725971112
2292,Ohio GOP Senate candidates pitch federal abortion bans even after voters protected reproductive rights,fts,2292,3,-9.7149595994016
452,"64K women and girls became pregnant due to rape in states with abortion bans, study estimates",fts,452,4,-9.163558569425538
9187,"Abortion bans drive away up to half of young talent, CNBC/Generation Lab youth survey finds",fts,9187,5,-9.163558569425538
11154,Meet the anti-abortion group using white coats and research to advance its cause,vec,11154,1,0.7592407464981079
3958,Supreme Court signals it is likely to reject a challenge to abortion pill access,vec,3958,2,0.8076152205467224
6011,Supreme Court wrestles with abortion clash over emergency room treatment for pregnant women,vec,6011,3,0.812921941280365
3933,The Supreme Court puts the GOP’s abortion dilemma back in the spotlight: From the Politics Desk,vec,3933,4,0.8234137296676636
7381,Where abortion rights could be on the ballot this fall: From the Politics Desk,vec,7381,5,0.8250945210456848


## FTS + Vector search: RFF

In [4]:
.param set  name alex

[no code]

In [9]:
.param set query abortion bans


.param set weight_fts 1.0
.param set weight_vec 1.0
.param set rrf_k 60
.param set k 10


with vec_matches as (
  select
    article_id,
    row_number() over (order by distance) as rank_number,
    distance
  from vec_headlines
  where
    headline_embedding match lembed(:query)
    and k = :k
  order by distance
),
fts_matches as (
  select
    rowid,
    row_number() over (order by rank) as rank_number,
    rank as score
  from fts_headlines
  where headline match :query
  limit :k
),
final as (
  select
    articles.id,
    articles.headline,
    vec_matches.rank_number as vec_rank,
    fts_matches.rank_number as fts_rank,
    coalesce(1.0 / (:rrf_k + fts_matches.rank_number), 0.0) * :weight_fts
    + coalesce(1.0 / (:rrf_k + vec_matches.rank_number), 0.0) * :weight_vec
      as combined_rank,
    vec_matches.distance as vec_distance,
    fts_matches.score as fts_score
  from fts_matches
  full outer join vec_matches on vec_matches.article_id = fts_matches.rowid
  join articles on articles.rowid = coalesce(fts_matches.rowid, vec_matches.article_id)
  order by combined_rank desc
)
select * from final;



id,headline,vec_rank,fts_rank,combined_rank,vec_distance,fts_score
9776,"States with abortion bans saw birth control prescriptions fall post-Dobbs, study finds",7,2,0.031054405392392875,0.837793231010437,-10.016316725971112
9187,"Abortion bans drive away up to half of young talent, CNBC/Generation Lab youth survey finds",8,5,0.030090497737556562,0.8381417989730835,-9.163558569425538
10098,Kamala Harris says abortion bans are creating 'a health care crisis',,1,0.01639344262295082,,-10.678829270936067
11154,Meet the anti-abortion group using white coats and research to advance its cause,1,,0.01639344262295082,0.7592407464981079,
3958,Supreme Court signals it is likely to reject a challenge to abortion pill access,2,,0.016129032258064516,0.8076152205467224,
2292,Ohio GOP Senate candidates pitch federal abortion bans even after voters protected reproductive rights,,3,0.015873015873015872,,-9.7149595994016
6011,Supreme Court wrestles with abortion clash over emergency room treatment for pregnant women,3,,0.015873015873015872,0.812921941280365,
452,"64K women and girls became pregnant due to rape in states with abortion bans, study estimates",,4,0.015625,,-9.163558569425538
3933,The Supreme Court puts the GOP’s abortion dilemma back in the spotlight: From the Politics Desk,4,,0.015625,0.8234137296676636,
7381,Where abortion rights could be on the ballot this fall: From the Politics Desk,5,,0.015384615384615385,0.8250945210456848,


### Re-rank by semantics

In [12]:
.param set query abortion bans
.param set k 10


with fts_matches as (
  select
    rowid,
    row_number() over (order by rank) as fts_rank_number,
    rank as score
  from fts_headlines
  where headline match :query
  limit :k
),
final as (
  select
    articles.id,
    articles.headline,
    fts_matches.*
  from fts_matches
  left join articles on articles.rowid = fts_matches.rowid
  order by vec_distance_cosine(lembed(:query), lembed(articles.headline))
)
select * from final;



id,headline,rowid,fts_rank_number,score
452,"64K women and girls became pregnant due to rape in states with abortion bans, study estimates",452,4,-9.163558569425538
9776,"States with abortion bans saw birth control prescriptions fall post-Dobbs, study finds",9776,2,-10.016316725971112
2292,Ohio GOP Senate candidates pitch federal abortion bans even after voters protected reproductive rights,2292,3,-9.7149595994016
10098,Kamala Harris says abortion bans are creating 'a health care crisis',10098,1,-10.678829270936067
9187,"Abortion bans drive away up to half of young talent, CNBC/Generation Lab youth survey finds",9187,5,-9.163558569425538
